# Import Libraries & Data

In [2]:
import os
import glob
import re
import numpy as np
import pandas as pd

In [3]:
dataset = pd.read_pickle('kaggle_openface_merged.pkl')

In [4]:
dataset

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,file_name,label,video_id
0,1,0,0.000,0.98,1,-0.265343,0.270043,-0.925565,-0.394123,0.281717,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie,trial_lie_001
1,2,0,0.033,0.98,1,-0.269314,0.280146,-0.921405,-0.399485,0.291082,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie,trial_lie_001
2,3,0,0.067,0.98,1,-0.276762,0.281786,-0.918694,-0.402257,0.294677,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie,trial_lie_001
3,4,0,0.100,0.98,1,-0.267571,0.284353,-0.920624,-0.390941,0.297649,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie,trial_lie_001
4,5,0,0.133,0.98,1,-0.251324,0.286230,-0.924613,-0.374932,0.299439,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_lie_001.csv,lie,trial_lie_001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91331,536,0,19.815,0.98,1,0.399038,0.150437,-0.904509,0.234647,0.184356,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth,trial_truth_060
91332,537,0,19.852,0.98,1,0.394478,0.167984,-0.903420,0.232950,0.200715,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth,trial_truth_060
91333,538,0,19.889,0.98,1,0.392632,0.175124,-0.902869,0.232472,0.207146,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth,trial_truth_060
91334,539,0,19.926,0.98,1,0.402144,0.140985,-0.904657,0.236606,0.176743,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,trial_truth_060.csv,truth,trial_truth_060


# Aggregate all data again

In [5]:
# at first we used only the AU column -> no good results so we added features for pose and gaze as well
#dataset.columns.values

In [6]:
dataset.sort_values(['video_id', 'face_id', 'timestamp'], inplace = True)

In [7]:
# keep only au, pose & gaze columns

au_cols = [c for c in dataset.columns if c.endswith("_r")]

pose_cols = [c for c in dataset.columns if c.startswith("pose_")]

gaze_cols = [c for c in dataset.columns if c.startswith("gaze_")]

final_columns = au_cols + pose_cols + gaze_cols
#final_columns

## Compute aggregates (mean/max/std) per video

In [8]:
# compute aggregates (mean/max/std) per video pt.1

# compute  mean per video
video_mean = (dataset.groupby('video_id')[final_columns].mean())
video_mean

# compute AU mean per video
video_max = (dataset.groupby('video_id')[final_columns].max())
video_max

# compute AU mean per video
video_std = (dataset.groupby('video_id')[final_columns].std())
video_std

,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,...,pose_Ry,pose_Rz,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,gaze_angle_y
video_id,,,,,,,,,,,,,,,,,,,,,
trial_lie_001,0.324179,0.163841,0.335093,0.071755,0.235783,0.411961,0.135490,0.242846,0.176749,0.029797,...,0.086833,0.080215,0.060920,0.022304,0.016984,0.051112,0.028465,0.020777,0.060778,0.025531
trial_lie_002,0.280204,0.265914,0.464621,0.173493,0.312652,0.378970,0.174106,0.391662,0.226734,0.224342,...,0.118626,0.082763,0.080422,0.036474,0.020077,0.067780,0.039030,0.029230,0.081053,0.038560
trial_lie_003,0.295436,0.208954,0.337934,0.073683,0.180666,0.449050,0.115039,0.249663,0.166174,0.032996,...,0.084538,0.042073,0.060152,0.014296,0.017629,0.049329,0.017080,0.020929,0.060432,0.013833
trial_lie_004,0.226205,0.188889,0.437078,0.101756,0.250258,0.350401,0.100998,0.290013,0.206197,0.029500,...,0.069979,0.047132,0.050194,0.029147,0.013175,0.037694,0.030538,0.016335,0.047490,0.030782
trial_lie_005,0.616221,0.462719,0.417614,0.198268,0.236141,0.390057,0.103947,0.388885,0.209974,0.021643,...,0.073683,0.074510,0.053387,0.029745,0.013700,0.042193,0.034177,0.016761,0.051721,0.032626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trial_truth_056,0.155541,0.123977,0.255314,0.160047,0.198990,0.246462,0.096145,0.134633,0.075287,0.238906,...,0.023709,0.027037,0.065080,0.052906,0.025180,0.056085,0.077427,0.043238,0.062136,0.073676
trial_truth_057,0.248095,0.214293,0.441438,0.140062,0.121267,0.506605,0.127586,0.384440,0.206622,0.763449,...,0.231240,0.052074,0.157110,0.032062,0.022844,0.160094,0.029692,0.018850,0.165719,0.033208
trial_truth_058,0.216975,0.175674,0.392246,0.114391,0.222232,0.483808,0.107181,0.275670,0.184555,0.128719,...,0.053284,0.058234,0.037338,0.023923,0.007844,0.032701,0.028071,0.011642,0.036896,0.026924


In [9]:
# compute aggregates (mean/max/std) per video pt.1

# compute  mean per video
video_mean = (dataset.groupby('video_id')[final_columns].mean().add_prefix("stau_mean_"))
video_mean

# compute AU mean per video
video_max = (dataset.groupby('video_id')[final_columns].max().add_prefix("stau_max_"))
video_max

# compute AU mean per video
video_std = (dataset.groupby('video_id')[final_columns].std().add_prefix("stau_std_"))
video_std

,stau_std_AU01_r,stau_std_AU02_r,stau_std_AU04_r,stau_std_AU05_r,stau_std_AU06_r,stau_std_AU07_r,stau_std_AU09_r,stau_std_AU10_r,stau_std_AU12_r,stau_std_AU14_r,...,stau_std_pose_Ry,stau_std_pose_Rz,stau_std_gaze_0_x,stau_std_gaze_0_y,stau_std_gaze_0_z,stau_std_gaze_1_x,stau_std_gaze_1_y,stau_std_gaze_1_z,stau_std_gaze_angle_x,stau_std_gaze_angle_y
video_id,,,,,,,,,,,,,,,,,,,,,
trial_lie_001,0.324179,0.163841,0.335093,0.071755,0.235783,0.411961,0.135490,0.242846,0.176749,0.029797,...,0.086833,0.080215,0.060920,0.022304,0.016984,0.051112,0.028465,0.020777,0.060778,0.025531
trial_lie_002,0.280204,0.265914,0.464621,0.173493,0.312652,0.378970,0.174106,0.391662,0.226734,0.224342,...,0.118626,0.082763,0.080422,0.036474,0.020077,0.067780,0.039030,0.029230,0.081053,0.038560
trial_lie_003,0.295436,0.208954,0.337934,0.073683,0.180666,0.449050,0.115039,0.249663,0.166174,0.032996,...,0.084538,0.042073,0.060152,0.014296,0.017629,0.049329,0.017080,0.020929,0.060432,0.013833
trial_lie_004,0.226205,0.188889,0.437078,0.101756,0.250258,0.350401,0.100998,0.290013,0.206197,0.029500,...,0.069979,0.047132,0.050194,0.029147,0.013175,0.037694,0.030538,0.016335,0.047490,0.030782
trial_lie_005,0.616221,0.462719,0.417614,0.198268,0.236141,0.390057,0.103947,0.388885,0.209974,0.021643,...,0.073683,0.074510,0.053387,0.029745,0.013700,0.042193,0.034177,0.016761,0.051721,0.032626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trial_truth_056,0.155541,0.123977,0.255314,0.160047,0.198990,0.246462,0.096145,0.134633,0.075287,0.238906,...,0.023709,0.027037,0.065080,0.052906,0.025180,0.056085,0.077427,0.043238,0.062136,0.073676
trial_truth_057,0.248095,0.214293,0.441438,0.140062,0.121267,0.506605,0.127586,0.384440,0.206622,0.763449,...,0.231240,0.052074,0.157110,0.032062,0.022844,0.160094,0.029692,0.018850,0.165719,0.033208
trial_truth_058,0.216975,0.175674,0.392246,0.114391,0.222232,0.483808,0.107181,0.275670,0.184555,0.128719,...,0.053284,0.058234,0.037338,0.023923,0.007844,0.032701,0.028071,0.011642,0.036896,0.026924


In [10]:
agg_1 = video_mean.merge(video_max, on = 'video_id')
agg_1

,stau_mean_AU01_r,stau_mean_AU02_r,stau_mean_AU04_r,stau_mean_AU05_r,stau_mean_AU06_r,stau_mean_AU07_r,stau_mean_AU09_r,stau_mean_AU10_r,stau_mean_AU12_r,stau_mean_AU14_r,...,stau_max_pose_Ry,stau_max_pose_Rz,stau_max_gaze_0_x,stau_max_gaze_0_y,stau_max_gaze_0_z,stau_max_gaze_1_x,stau_max_gaze_1_y,stau_max_gaze_1_z,stau_max_gaze_angle_x,stau_max_gaze_angle_y
video_id,,,,,,,,,,,,,,,,,,,,,
trial_lie_001,0.218902,0.075882,0.908216,0.032902,0.165941,0.395275,0.065431,0.179059,0.404059,0.007353,...,0.758,0.245,-0.126530,0.286230,-0.880234,-0.270586,0.305706,-0.820203,-0.210,0.314
trial_lie_002,0.189193,0.117258,0.922272,0.068461,0.357739,0.361887,0.087916,0.652170,0.337622,0.118985,...,0.754,0.256,-0.000191,0.379623,-0.854209,-0.162407,0.369305,-0.787073,-0.084,0.408
trial_lie_003,0.196792,0.099670,1.321368,0.033113,0.084811,0.401179,0.071415,0.332453,0.345330,0.006321,...,0.710,-0.033,-0.188996,0.309133,-0.865997,-0.338230,0.320483,-0.801469,-0.283,0.348
trial_lie_004,0.141662,0.080774,1.166361,0.046705,0.222063,0.485931,0.051576,0.550172,0.340229,0.006160,...,0.686,-0.003,-0.198591,0.344970,-0.881795,-0.358915,0.350354,-0.818652,-0.294,0.383
trial_lie_005,0.367677,0.204531,1.336260,0.075233,0.262719,0.419681,0.050153,0.531812,0.336477,0.003516,...,0.782,0.382,-0.181269,0.355966,-0.866045,-0.327828,0.359307,-0.799895,-0.271,0.389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trial_truth_056,0.099715,0.060029,1.685044,0.054455,0.460608,0.228302,0.036850,0.101541,0.020000,0.275015,...,0.545,-0.021,-0.045807,0.566883,-0.815605,-0.276047,0.564494,-0.655025,-0.199,0.640
trial_truth_057,0.166471,0.086874,0.633898,0.057217,0.047024,0.797234,0.062678,1.007603,0.352546,1.508771,...,0.519,0.142,0.449427,0.278473,-0.874838,0.313031,0.263782,-0.875499,0.399,0.284
trial_truth_058,0.149254,0.085683,0.704968,0.048921,0.298302,0.771810,0.053079,1.052524,0.546286,0.067063,...,0.565,0.208,-0.063102,0.300141,-0.925767,-0.233710,0.311686,-0.874459,-0.155,0.319


In [11]:
agg_2 = agg_1.merge(video_std, on = 'video_id')
agg_2

,stau_mean_AU01_r,stau_mean_AU02_r,stau_mean_AU04_r,stau_mean_AU05_r,stau_mean_AU06_r,stau_mean_AU07_r,stau_mean_AU09_r,stau_mean_AU10_r,stau_mean_AU12_r,stau_mean_AU14_r,...,stau_std_pose_Ry,stau_std_pose_Rz,stau_std_gaze_0_x,stau_std_gaze_0_y,stau_std_gaze_0_z,stau_std_gaze_1_x,stau_std_gaze_1_y,stau_std_gaze_1_z,stau_std_gaze_angle_x,stau_std_gaze_angle_y
video_id,,,,,,,,,,,,,,,,,,,,,
trial_lie_001,0.218902,0.075882,0.908216,0.032902,0.165941,0.395275,0.065431,0.179059,0.404059,0.007353,...,0.086833,0.080215,0.060920,0.022304,0.016984,0.051112,0.028465,0.020777,0.060778,0.025531
trial_lie_002,0.189193,0.117258,0.922272,0.068461,0.357739,0.361887,0.087916,0.652170,0.337622,0.118985,...,0.118626,0.082763,0.080422,0.036474,0.020077,0.067780,0.039030,0.029230,0.081053,0.038560
trial_lie_003,0.196792,0.099670,1.321368,0.033113,0.084811,0.401179,0.071415,0.332453,0.345330,0.006321,...,0.084538,0.042073,0.060152,0.014296,0.017629,0.049329,0.017080,0.020929,0.060432,0.013833
trial_lie_004,0.141662,0.080774,1.166361,0.046705,0.222063,0.485931,0.051576,0.550172,0.340229,0.006160,...,0.069979,0.047132,0.050194,0.029147,0.013175,0.037694,0.030538,0.016335,0.047490,0.030782
trial_lie_005,0.367677,0.204531,1.336260,0.075233,0.262719,0.419681,0.050153,0.531812,0.336477,0.003516,...,0.073683,0.074510,0.053387,0.029745,0.013700,0.042193,0.034177,0.016761,0.051721,0.032626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trial_truth_056,0.099715,0.060029,1.685044,0.054455,0.460608,0.228302,0.036850,0.101541,0.020000,0.275015,...,0.023709,0.027037,0.065080,0.052906,0.025180,0.056085,0.077427,0.043238,0.062136,0.073676
trial_truth_057,0.166471,0.086874,0.633898,0.057217,0.047024,0.797234,0.062678,1.007603,0.352546,1.508771,...,0.231240,0.052074,0.157110,0.032062,0.022844,0.160094,0.029692,0.018850,0.165719,0.033208
trial_truth_058,0.149254,0.085683,0.704968,0.048921,0.298302,0.771810,0.053079,1.052524,0.546286,0.067063,...,0.053284,0.058234,0.037338,0.023923,0.007844,0.032701,0.028071,0.011642,0.036896,0.026924


In [12]:
video_dau = (
    dataset
    .groupby("video_id")[final_columns]
    .apply(lambda df: np.nanmean(np.abs(np.diff(df.values, axis=0)), axis=0))
    .apply(pd.Series)
    .set_axis([f"movaver_{c}" for c in final_columns], axis=1)
)

video_dau

,movaver_AU01_r,movaver_AU02_r,movaver_AU04_r,movaver_AU05_r,movaver_AU06_r,movaver_AU07_r,movaver_AU09_r,movaver_AU10_r,movaver_AU12_r,movaver_AU14_r,...,movaver_pose_Ry,movaver_pose_Rz,movaver_gaze_0_x,movaver_gaze_0_y,movaver_gaze_0_z,movaver_gaze_1_x,movaver_gaze_1_y,movaver_gaze_1_z,movaver_gaze_angle_x,movaver_gaze_angle_y
video_id,,,,,,,,,,,,,,,,,,,,,
trial_lie_001,0.035285,0.026405,0.061179,0.014735,0.034303,0.076739,0.016660,0.031965,0.040452,0.004322,...,0.012251,0.007827,0.007940,0.004800,0.002726,0.007072,0.004653,0.003578,0.008246,0.005100
trial_lie_002,0.034888,0.027925,0.064594,0.016578,0.045059,0.066636,0.016738,0.060316,0.036000,0.021096,...,0.009067,0.007768,0.005491,0.005505,0.002316,0.005119,0.005725,0.003160,0.005934,0.006347
trial_lie_003,0.040995,0.025308,0.048199,0.016114,0.021422,0.093602,0.010190,0.036919,0.037915,0.003128,...,0.012692,0.005858,0.008720,0.004067,0.002913,0.007564,0.004032,0.003589,0.009009,0.004261
trial_lie_004,0.029253,0.027644,0.054856,0.013908,0.032989,0.088822,0.014310,0.049626,0.031178,0.002299,...,0.005805,0.006006,0.004314,0.004859,0.001755,0.003490,0.004883,0.002139,0.004296,0.005425
trial_lie_005,0.032120,0.024253,0.057356,0.012452,0.035907,0.066315,0.010358,0.045587,0.032184,0.001622,...,0.006167,0.005577,0.003932,0.004017,0.001547,0.003473,0.004005,0.001968,0.004023,0.004445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trial_truth_056,0.024568,0.021690,0.044411,0.015403,0.037112,0.065265,0.010373,0.022927,0.005206,0.041994,...,0.002851,0.002135,0.010457,0.010144,0.004509,0.010491,0.011107,0.007138,0.009094,0.011209
trial_truth_057,0.030299,0.022724,0.057373,0.025343,0.011142,0.099069,0.015510,0.070018,0.034807,0.084148,...,0.007343,0.005591,0.005000,0.004181,0.001699,0.004782,0.004334,0.001259,0.005081,0.004504
trial_truth_058,0.031574,0.025040,0.054722,0.015946,0.045723,0.090954,0.013068,0.065008,0.035644,0.020684,...,0.006525,0.006035,0.004349,0.004510,0.001251,0.003960,0.004558,0.001692,0.004297,0.004789


In [13]:
agg_3 = agg_2.merge(video_dau, on = 'video_id')
agg_3

,stau_mean_AU01_r,stau_mean_AU02_r,stau_mean_AU04_r,stau_mean_AU05_r,stau_mean_AU06_r,stau_mean_AU07_r,stau_mean_AU09_r,stau_mean_AU10_r,stau_mean_AU12_r,stau_mean_AU14_r,...,movaver_pose_Ry,movaver_pose_Rz,movaver_gaze_0_x,movaver_gaze_0_y,movaver_gaze_0_z,movaver_gaze_1_x,movaver_gaze_1_y,movaver_gaze_1_z,movaver_gaze_angle_x,movaver_gaze_angle_y
video_id,,,,,,,,,,,,,,,,,,,,,
trial_lie_001,0.218902,0.075882,0.908216,0.032902,0.165941,0.395275,0.065431,0.179059,0.404059,0.007353,...,0.012251,0.007827,0.007940,0.004800,0.002726,0.007072,0.004653,0.003578,0.008246,0.005100
trial_lie_002,0.189193,0.117258,0.922272,0.068461,0.357739,0.361887,0.087916,0.652170,0.337622,0.118985,...,0.009067,0.007768,0.005491,0.005505,0.002316,0.005119,0.005725,0.003160,0.005934,0.006347
trial_lie_003,0.196792,0.099670,1.321368,0.033113,0.084811,0.401179,0.071415,0.332453,0.345330,0.006321,...,0.012692,0.005858,0.008720,0.004067,0.002913,0.007564,0.004032,0.003589,0.009009,0.004261
trial_lie_004,0.141662,0.080774,1.166361,0.046705,0.222063,0.485931,0.051576,0.550172,0.340229,0.006160,...,0.005805,0.006006,0.004314,0.004859,0.001755,0.003490,0.004883,0.002139,0.004296,0.005425
trial_lie_005,0.367677,0.204531,1.336260,0.075233,0.262719,0.419681,0.050153,0.531812,0.336477,0.003516,...,0.006167,0.005577,0.003932,0.004017,0.001547,0.003473,0.004005,0.001968,0.004023,0.004445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trial_truth_056,0.099715,0.060029,1.685044,0.054455,0.460608,0.228302,0.036850,0.101541,0.020000,0.275015,...,0.002851,0.002135,0.010457,0.010144,0.004509,0.010491,0.011107,0.007138,0.009094,0.011209
trial_truth_057,0.166471,0.086874,0.633898,0.057217,0.047024,0.797234,0.062678,1.007603,0.352546,1.508771,...,0.007343,0.005591,0.005000,0.004181,0.001699,0.004782,0.004334,0.001259,0.005081,0.004504
trial_truth_058,0.149254,0.085683,0.704968,0.048921,0.298302,0.771810,0.053079,1.052524,0.546286,0.067063,...,0.006525,0.006035,0.004349,0.004510,0.001251,0.003960,0.004558,0.001692,0.004297,0.004789


In [14]:
data_temp = (dataset[['video_id', 'face_id', 'label']].drop_duplicates(subset = 'video_id'))
data_temp

,video_id,face_id,label
0,trial_lie_001,0,lie
510,trial_lie_002,0,lie
2384,trial_lie_003,0,lie
2596,trial_lie_004,0,lie
2945,trial_lie_005,0,lie
...,...,...,...
87198,trial_truth_056,0,truth
88217,trial_truth_057,0,truth
89356,trial_truth_058,0,truth
89986,trial_truth_059,0,truth


In [15]:
movaver_dataset = agg_3.merge(data_temp, on = 'video_id')
movaver_dataset

,video_id,stau_mean_AU01_r,stau_mean_AU02_r,stau_mean_AU04_r,stau_mean_AU05_r,stau_mean_AU06_r,stau_mean_AU07_r,stau_mean_AU09_r,stau_mean_AU10_r,stau_mean_AU12_r,...,movaver_gaze_0_x,movaver_gaze_0_y,movaver_gaze_0_z,movaver_gaze_1_x,movaver_gaze_1_y,movaver_gaze_1_z,movaver_gaze_angle_x,movaver_gaze_angle_y,face_id,label
0,trial_lie_001,0.218902,0.075882,0.908216,0.032902,0.165941,0.395275,0.065431,0.179059,0.404059,...,0.007940,0.004800,0.002726,0.007072,0.004653,0.003578,0.008246,0.005100,0,lie
1,trial_lie_002,0.189193,0.117258,0.922272,0.068461,0.357739,0.361887,0.087916,0.652170,0.337622,...,0.005491,0.005505,0.002316,0.005119,0.005725,0.003160,0.005934,0.006347,0,lie
2,trial_lie_003,0.196792,0.099670,1.321368,0.033113,0.084811,0.401179,0.071415,0.332453,0.345330,...,0.008720,0.004067,0.002913,0.007564,0.004032,0.003589,0.009009,0.004261,0,lie
3,trial_lie_004,0.141662,0.080774,1.166361,0.046705,0.222063,0.485931,0.051576,0.550172,0.340229,...,0.004314,0.004859,0.001755,0.003490,0.004883,0.002139,0.004296,0.005425,0,lie
4,trial_lie_005,0.367677,0.204531,1.336260,0.075233,0.262719,0.419681,0.050153,0.531812,0.336477,...,0.003932,0.004017,0.001547,0.003473,0.004005,0.001968,0.004023,0.004445,0,lie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,trial_truth_056,0.099715,0.060029,1.685044,0.054455,0.460608,0.228302,0.036850,0.101541,0.020000,...,0.010457,0.010144,0.004509,0.010491,0.011107,0.007138,0.009094,0.011209,0,truth
116,trial_truth_057,0.166471,0.086874,0.633898,0.057217,0.047024,0.797234,0.062678,1.007603,0.352546,...,0.005000,0.004181,0.001699,0.004782,0.004334,0.001259,0.005081,0.004504,0,truth
117,trial_truth_058,0.149254,0.085683,0.704968,0.048921,0.298302,0.771810,0.053079,1.052524,0.546286,...,0.004349,0.004510,0.001251,0.003960,0.004558,0.001692,0.004297,0.004789,0,truth
118,trial_truth_059,0.253333,0.099136,0.400802,0.074395,0.448728,0.471444,0.080543,0.915852,0.703444,...,0.008798,0.005402,0.001586,0.008163,0.005242,0.002507,0.008868,0.005581,0,truth


In [16]:
movaver_dataset.columns.values

array(['video_id', 'stau_mean_AU01_r', 'stau_mean_AU02_r',
       'stau_mean_AU04_r', 'stau_mean_AU05_r', 'stau_mean_AU06_r',
       'stau_mean_AU07_r', 'stau_mean_AU09_r', 'stau_mean_AU10_r',
       'stau_mean_AU12_r', 'stau_mean_AU14_r', 'stau_mean_AU15_r',
       'stau_mean_AU17_r', 'stau_mean_AU20_r', 'stau_mean_AU23_r',
       'stau_mean_AU25_r', 'stau_mean_AU26_r', 'stau_mean_AU45_r',
       'stau_mean_pose_Tx', 'stau_mean_pose_Ty', 'stau_mean_pose_Tz',
       'stau_mean_pose_Rx', 'stau_mean_pose_Ry', 'stau_mean_pose_Rz',
       'stau_mean_gaze_0_x', 'stau_mean_gaze_0_y', 'stau_mean_gaze_0_z',
       'stau_mean_gaze_1_x', 'stau_mean_gaze_1_y', 'stau_mean_gaze_1_z',
       'stau_mean_gaze_angle_x', 'stau_mean_gaze_angle_y',
       'stau_max_AU01_r', 'stau_max_AU02_r', 'stau_max_AU04_r',
       'stau_max_AU05_r', 'stau_max_AU06_r', 'stau_max_AU07_r',
       'stau_max_AU09_r', 'stau_max_AU10_r', 'stau_max_AU12_r',
       'stau_max_AU14_r', 'stau_max_AU15_r', 'stau_max_AU17_r',
     

## Compute temporal aggregates (mean/max/var) per video

In [17]:
# # compute aggregates (mean/max/std) per video pt.2

# grouped = dataset.groupby(['video_id', 'face_id', 'label'], sort = False)

# # create temporal diff for AU columns
# for c in au_cols:
#     dataset[f"dau_{c}"] = grouped[c].diff().fillna(0)
#     dataset[f"adau_{c}"] = grouped[f"dau_{c}"].transform(np.abs)

# # create temporal diff for gaze/pose columns
# for c in au_cols:
#     dataset[f"dgp_{c}"] = grouped[c].diff().fillna(0)
#     dataset[f"adgp_{c}"] = grouped[f"dgp_{c}"].transform(np.abs)

# dataset

In [18]:
#dataset.columns.values

In [19]:
# # keep only diff_au, adiff_au, diff_gp & adiff_gp columns

# diff_au_cols = [c for c in dataset.columns if c.startswith('dau_')]

# adiff_au_cols = [c for c in dataset.columns if c.startswith('adau_')]

# diff_gp_cols = [c for c in dataset.columns if c.startswith('dgp_')]

# adiff_gp_cols = [c for c in dataset.columns if c.startswith('adgp_')]

# #final_columns = au_cols + pose_cols + pose_cols
# #final_columns

In [20]:
# # final step: group by & aggregate

# # AU diffs
# dau_mean = dataset.groupby(['video_id', 'face_id', 'label'])[diff_au_cols].mean().add_prefix("dau_mean_")
# dau_min = dataset.groupby(['video_id', 'face_id', 'label'])[diff_au_cols].min().add_prefix("dau_min_")
# dau_max = dataset.groupby(['video_id', 'face_id', 'label'])[diff_au_cols].max().add_prefix("dau_max_")
# dau_var = dataset.groupby(['video_id', 'face_id', 'label'])[diff_au_cols].var().add_prefix("dau_var_")

# # AU absolute diffs
# adau_mean = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_au_cols].mean().add_prefix("adau_mean_")
# adau_min = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_au_cols].min().add_prefix("adau_min_")
# adau_max = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_au_cols].max().add_prefix("adau_max_")
# adau_var = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_au_cols].var().add_prefix("adau_var_")

# # gaze & pose diffs
# dgp_mean = dataset.groupby(['video_id', 'face_id', 'label'])[diff_gp_cols].mean().add_prefix("dgp_mean_")
# dgp_min = dataset.groupby(['video_id', 'face_id', 'label'])[diff_gp_cols].min().add_prefix("dgp_min_")
# dgp_max = dataset.groupby(['video_id', 'face_id', 'label'])[diff_gp_cols].max().add_prefix("dgp_max_")
# dgp_var = dataset.groupby(['video_id', 'face_id', 'label'])[diff_gp_cols].var().add_prefix("dgp_var_")

# # gaze & pose absolute diffs
# adgp_mean = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_gp_cols].mean().add_prefix("adgp_mean_")
# adgp_min = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_gp_cols].min().add_prefix("adgp_min_")
# adgp_max = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_gp_cols].max().add_prefix("adgp_max_")
# adgp_var = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_gp_cols].var().add_prefix("adgp_var_")

In [21]:
# # union all aggregates in one table at last

# agg_blocks = [
#     dau_mean, dau_min, dau_max, dau_var, # AU diffs
#     adau_mean, adau_min, adau_max, adau_var,  # AU absolute diffs
#     dgp_mean, dgp_min, dgp_max, dgp_var,   # gaze & pose diffs
#     adgp_mean, adgp_min, adgp_max, adgp_var # gaze & pose absolute diffs
# ]

# for agg in agg_blocks:
#     if agg.empty:
#         continue
#     data_temp = data_temp.merge(agg.reset_index(), on = ['video_id', 'face_id', 'label'], how = 'left')

# Output Files

#### ---> From now on we have one row per video with AU, Pose, Gaze mean/max/std & label

In [22]:
#data_temp.columns

In [23]:
movaver_dataset.to_pickle('kaggle_movaver_dataset.pkl')